## Project Template

In [10]:
from pyspark.sql.functions import explode
from pyspark.sql.functions import split

from pyspark.context import SparkContext
from pyspark.conf import SparkConf
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import col

spark = SparkSession.builder \
    .appName("MyApp") \
    .config("spark.jars.packages", 'org.apache.spark:spark-sql-kafka-0-10_2.12:3.0.0') \
    .config("spark.sql.repl.eagerEval.enabled", True) \
    .getOrCreate()


### Starting the Kafka stream

Be sure to start the stream on Kafka!

In [3]:
from confluent_kafka import SerializingProducer, DeserializingConsumer
from confluent_kafka.serialization import StringSerializer, StringDeserializer
from confluent_kafka.admin import AdminClient, NewTopic
from uuid import uuid4
import sys, random

brokers = "kafka1:9092,kafka2:9093"

In [4]:
pconf = {
    'bootstrap.servers': brokers,
    'partitioner': 'murmur2_random',
    'key.serializer': StringSerializer('utf_8'),
    'value.serializer':  StringSerializer('utf_8')
}

In [5]:
p = SerializingProducer(pconf)

In [6]:
taxies = "sample.csv"

In [7]:
from datetime import datetime  
  #2023-10-13T08:16:13Z
def construct_ride(row):
    time_stamp = time.time()
    date_time = datetime.fromtimestamp(time_stamp)
    str_date_time = date_time.strftime("%Y-%m-%dT%H:%M:%SZ") #"%d-%m-%Y, %H:%M:%S"
    ride = {"vendor_id": row[2],
            "pickup_datetime": row[5],
            "droppff_datetime":row[5],
            "pickup_longitude": float(row[10]),
            "pickup_latitude": float(row[11]),
            "dropoff_longitude": float(row[12]),
            "dropoff_latitude": float(row[13]),
             "timestamp":str_date_time
             }
    return ride

In [9]:
import csv, json
import time
n = 0
with open(taxies) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    next(csv_reader, None) # Skipping the headers
    try:
        for row in csv_reader:
            #if n == 10:
             #  break
            ride = construct_ride(row)
            #print(ride)
            p.produce('ride', value=json.dumps(ride))
            p.poll(0)
            #p.flush()
            time.sleep(0.5)
            #n = n + 1
    except BufferError:
        sys.stderr.write('%% Local producer queue is full (%d messages awaiting delivery): try again\n' % len(p))


KeyboardInterrupt: 

### Reading the stream

In [11]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, BooleanType, TimestampType, DateType
# TO MODIFY FOR YOUR SCHEMA
schema = StructType(
      [
        StructField("vendor_id", StringType(), False),
        StructField("pickup_datetime", StringType(), False),
        StructField("droppff_datetime", StringType(), False),
        StructField("pickup_longitude", DoubleType(), False),
        StructField("pickup_latitude", DoubleType(), False),
        StructField("dropoff_longitude", DoubleType(), False),
        StructField("dropoff_latitude", DoubleType(), False),
        StructField("start_borough", StringType(), False),
        StructField("end_borough", StringType(), False),
        StructField("timestamp", TimestampType(), False),
      ]
    )

In [12]:
kafka_server = "kafka1:9092"   
from pyspark.sql.functions import from_json

lines = (spark.readStream                        # Get the DataStreamReader
  .format("kafka")                                 # Specify the source format as "kafka"
  .option("kafka.bootstrap.servers", kafka_server) # Configure the Kafka server name and port
  .option("subscribe", "stock")                       # Subscribe to the "en" Kafka topic 
  .option("startingOffsets", "earliest")           # The start point when a query is started
  .option("maxOffsetsPerTrigger", 100)             # Rate limit on max offsets per trigger interval
  .load()
# Load the DataFrame
)
df = lines.select("parsed_value.*")


AnalysisException: [CANNOT_RESOLVE_STAR_EXPAND] Cannot resolve `parsed_value`.* given input columns . Please check that the specified table or struct exists and is accessible in the input columns.

In [ ]:
lines.isStreaming

## The project starts here

You can create a

## [Query 1] Utilization over a window of 5, 10, and 15 minutes per taxi/driver. This can be computed by computing the idle time per taxi. How does it change? Is there an optimal window?

## [Query 2] The average time it takes for a taxi to find its next fare(trip) per destination borough. This can be computed by finding the time difference, e.g. in seconds, between the trip's drop off and the next trip's pick up within a given unit of time

In [ ]:
# remember you can register another stream


## [Query 3] The number of trips that started and ended within the same borough in the last hour

In [ ]:
# remember you can register another stream


## [Query 4] The number of trips that started in one borough and ended in another one in the last hour